In [ ]:
import csv
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import textwrap

In [ ]:
# Find repo root even if notebook is inside /notebooks
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
def analyze_csv_ranges(file_path,delim=","):
    """
    Analyze a CSV file to separate columns and find ranges for all columns.
    For numeric columns: calculates min, max, range
    For text columns: shows unique count and sample values
    """
    
    # Read the CSV file
    try:
        df = pd.read_csv(file_path,delimiter=delim)
    except Exception as e:
        print(f"Error reading file: {e}")
        return
    
    print(df.describe())
    print("-" * 80)
    print(f"File: {file_path}")
    print(f"Total rows: {len(df)}")
    print(f"Total columns: {len(df.columns)}")
    print("-" * 80)


    for i, column in enumerate(df.columns):
        print("-" * 80)
        print(f"Column {i}: {column}")
        print("-" * 80)
        col_data = df[column]
        first_data = 0
        for n in range(len(col_data)):
            if pd.isna(col_data[n]):
                first_data = n
            else: break

        if pd.api.types.is_numeric_dtype(col_data): # Check if data is numeric
            clean_data = col_data.dropna()
            if len(clean_data)>0:
                min_val = clean_data.min()
                max_val = clean_data.max()
                print(f"Minimim Value: {min_val}")
                print(f"Maximum Value: {max_val}")
                print(f"Mean: {clean_data.mean():.2f}")
            else:
                print("All NaN or empty")
        else: 
            print("Type: Text/Categorcal")

        print(f"Preview of Column:")
        print(col_data[:3])
        print(col_data[-3:])
        print(f"Total values: {len(col_data)}")
        print(f"Unique values: {col_data.nunique()}")
        print(f"Missing values: {col_data.isna().sum()}")
        print(f'First value at: row number: {first_data}, on the {df["ts"][first_data]}')
        print(f"Fraction of No Readings {col_data.isna().sum()/len(col_data)} ")

    
    return df

In [ ]:
df = analyze_csv_ranges(DATA_DIR / 'combined_data.csv')

In [ ]:
df.hist(figsize=(12, 12), bins=20)

In [ ]:
df.drop(columns='ts').corr()

### **Data Frame with Positive Generation only**

In [ ]:
#dataframe with no zeros, i.e. positive or zero generation only

df_positive_generation =  analyze_csv_ranges(DATA_DIR / 'combined_data_positive_gen.csv')

In [ ]:
axes = df_positive_generation.hist(figsize=(12, 12), bins=20)

for ax in axes.flatten():
    title = ax.get_title()
    
    # Wrap title after certain character width
    wrapped_title = "\n".join(textwrap.wrap(title, width=20))
    
    ax.set_title(wrapped_title, fontsize=11)
    #ax.set_xlabel("Power [kW]", fontsize=10)
    ax.set_ylabel("Frequency", fontsize=10)
    
    # Improve tick readability
    ax.tick_params(axis='both', labelsize=9)

plt.tight_layout()
plt.show()

In [ ]:
df_positive_generation.drop(columns='ts').corr()

In [ ]:
analyze_csv_ranges(DATA_DIR / f'SyslabWind_15min.csv')

### I would chose the Aircon instead of Gaia, since Aircon has 10% no readings and Gaia has 25% no readings.
larger breaks: 
- July 30 - Aug 4
- Aug 8-22
- Oct 16-17
- and every once in a while theres a hole for a few hours 

In [ ]:
analyze_csv_ranges(DATA_DIR/'SyslabWeather_15min.csv')

### **SOLETE Data**

'The dataset includes 15 months of measurements from the 1st June 2018 to 1st September 2019 covering: Timestamp, air temperature, relative humidity, pressure, wind speed, wind direction, global horizontal irradiance, plane of array irradiance, and active power recorded from an 11 kW Gaia wind turbine and a 10 kW PV inverter.'

[1] SOLETE, a 15-month long holistic dataset including: Meteorology, co-located wind and solar PV power from Denmark with various resolutions (https://www.sciencedirect.com/science/article/pii/S2352340922002578#bib0002)

In [ ]:
dfSOLETE = analyze_csv_ranges(DATA_DIR / 'SOLETE_1sec_15min.csv') 

In [ ]:
dfSOLETE.drop(columns='ts').corr()


In [ ]:
axes = dfSOLETE.hist(figsize=(12, 12), bins=20)

for ax in axes.flatten():
    title = ax.get_title()
    
    # Wrap title after certain character width
    wrapped_title = "\n".join(textwrap.wrap(title, width=20))
    
    ax.set_title(wrapped_title, fontsize=11)
    #ax.set_xlabel("Power [kW]", fontsize=10)
    ax.set_ylabel("Frequency", fontsize=10)
    
    # Improve tick readability
    ax.tick_params(axis='both', labelsize=9)

plt.tight_layout()
plt.show()

Comments

- What columns are mostly 0 or empty?

No empty columns or rows. 

Power Gaia, Azimuth and Elevation have all 0 quantiles (25%, 50%, 75%)

- If there any, where are the big gaps in data?

There's no gaps in the data, only several occassions when the recorded data is 0. 

- How does this Gaia compare to the bigger dataframe's Gaia?

The data on the Gaia turbine first downloaded in SyslabWind.csv has 25% of gaps while SOLETE's has no gaps

Still, in SOLETE, the median is 0.0000 and the mean is 0.0002 with a max value of 10.552675555555554, which is close to the rated power (11 kW). Thus, it's a single or few outliers that show relevant power outputs, otherwise the data is irrelevant at 0 kW. It's correlation with wind speed is also corcerning at a mere 0.038058. :(((

- What does the solar production look like?

It has a strong correlation with the two irradiation types recorded: GHI (0.921497) and POA (0.998715)

The documentation on SOLETE states that the inverter has a power capacity of 10 kW, yet the following are the recorded values for the PV's output: 

median: 0.072028; mean: 1.024447; 75% quantile: 1.313590; max: 7.098779

It performs better than the PV unit in B715, yet it still seems to underperform :(

- Is there a correlation between the solar resource and the production?

Strong correlation between the two types of solar irradiation (GHI and POA Irr) and the PV Power Production ('Power Solar')


### **DTU Buildings Data**

In [ ]:
analyze_csv_ranges(DATA_DIR/ 'DTU_B325_B329_B329A_15min.csv')

Comments of building data:
- When are some of the most evident gaps?

We have data for the following periods: 

2021-02-28 23:00:00 until 2021-03-27 23:45:00 for all data sets

2021-03-28 00:15:00 until for 2021-03-28 00:45:00 for cols 6, 7, and 11

2021-03-28 01:00:00 until 2021-03-30 21:45:00 for all cols

**SMALL GAP**

2021-04-05 22:00:00 until 2021-04-28 21:15:00 for all cols

2021-04-28 21:30:00 until 2021-04-28 21:45:00 for cols 8 and 24

2021-04-28 22:00:00 until 2021-04-30 08:45:00 for most cols

2021-04-30 22:00:00 until 2021-05-05 21:45:00 for all cols

**BIGGER GAP**

2021-10-19 22:00:00 until 2021-10-20 21:45:00 for all cols

2021-10-21 22:00:00 until 2021-10-25 21:45:00 for all cols

2021-10-25 22:00:00 until 2021-10-26 21:45:00 only for col 2

2021-10-26 22:00:00 until 2021-10-30 22:45:00 for all cols

2021-10-31 01:00:00 until 2021-11-03 17:30:00 for most cols

**SMALL GAP**

2021-11-03 19:30:00 until 2021-11-04 22:45:00 for most cols

2021-11-04 23:00:00 until 2021-11-05 22:45:00 only for col 2

2021-11-05 23:00:00 until 2021-11-06 22:45:00 for all cols

2021-11-06 23:00:00 until 2021-11-07 21:45:00 only for col 2

2021-11-07 23:00:00 until 2021-11-11 20:15:00 for most cols 

2021-11-11 23:00:00 until 2021-11-12 17:30:00 for most cols 

2021-11-12 23:00:00 until 2021-11-13 17:15:00 for most cols 

2021-11-13 23:00:00 until 2021-11-14 15:00:00 for most cols

2021-11-14 23:00:00 until 2021-11-15 12:30:00 for most cols

2021-11-16 13:30:00 until 2021-11-16 17:00:00 only for col 23

2021-11-16 19:15:00 until 2021-11-18 05:15:00 for most cols 

2021-11-18 13:15:00 until 2021-11-18 21:00:00 for most cols 

2021-11-19 05:30:00 until 2022-01-12 01:45:00 for most cols

**BIGGER GAP**

2022-01-12 19:15:00 until 2022-01-12 20:45:00 only for col 17

2022-01-12 21:00:00 until 2022-02-03 13:45:00 for most cols 

**SMALL GAP**

2022-02-03 18:45:00 until  for most cols 

=> one hour gap 2022-03-26 00:00:00-2022-03-27 01:00:00

2022-03-27 01:00:00 until 2022-04-30 21:45:00 for most cols 

**SMALL GAP**

2022-04-30 22:00:00 until 2022-05-01 21:45:00 only for col 2

2022-05-01 22:00:00 until 2022-05-18 17:15:00 for most cols 

**SMALL GAP**

2022-05-18 22:00:00 until 2022-06-01 21:45:00 for most cols

**SMALL GAP**

2022-06-02 22:00:00 until 2022-06-20 14:15:00 for most cols 

**SMALL GAP**

2022-06-24 03:30:00 until 2022-07-05 21:45:00 for most cols 

**SMALL GAP**

2022-07-05 22:00:00 until 2022-07-06 21:45:00 only for col 2

- How is this data relevant?

Data on buildings B415, B325, B329, B346, B349

The total consumption of B325 is registered as well as that of its mixing plant total energy consumption and the consumption of specific rooms (R: 329, 961, 901B (two meters), 915, 951)

- Did you look into the naming legend linked in Energy Data Hub?

The link where the naming of the sensors is supposed to be detailed does not work. I also can't find anything with BMS on the current DTU website. Maybe we can request the data?
